<h1> Home Credit Default Risk Solution: Part 2</h1>

<div style="clear:both"></div>

<br>



# 1. PACKAGES

In [ ]:
# 1. PACKAGES
# -----------------------------------------------------------
# Base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Model
from lightgbm import LGBMClassifier

# Configuration
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# 2. DATA

In [ ]:
df = pd.read_feather("../input/homecredit-default-risk-step-by-step-1st-notebook/applications_traintest_feather")
pos = pd.read_feather("../input/homecredit-default-risk-step-by-step-1st-notebook/poscashbalance_agg_feather")
bb = pd.read_feather("../input/homecredit-default-risk-step-by-step-1st-notebook/bureau_bureaubalance_agg_feather")
cc = pd.read_feather("../input/homecredit-default-risk-step-by-step-1st-notebook/cc_feather")
ins = pd.read_feather("../input/homecredit-default-risk-step-by-step-1st-notebook/installments_payments_agg_feather")
prev = pd.read_feather("../input/homecredit-default-risk-step-by-step-1st-notebook/previous_applications_agg_feather")

print(df.shape, pos.shape, bb.shape, cc.shape, ins.shape, prev.shape)

for i in [pos, bb, cc, ins, prev]:
    df = pd.merge(df, i , how = "left", on = "SK_ID_CURR")
    
print(df.shape)

del pos, bb, ins, cc, prev

(356249, 145) (337252, 23) (305811, 456) (103558, 117) (339587, 49) (338857, 378)
(356249, 1163)


# 3. TRAIN-TEST SPLIT

In [ ]:
# Train-Test Split
df.columns = list(map(lambda x: str(x).replace(" ", "_").replace("-", "_").replace("_/_", "_").upper(), df.columns))
import re
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))


train = df[df.TARGET.isnull() == False]
test = df[df.TARGET.isnull()]

x_train = train.drop(["TARGET", "SK_ID_CURR"], axis = 1)
x_test = test.drop(["TARGET", "SK_ID_CURR"], axis = 1)
y_train = train.TARGET

# 4. MODEL

In [ ]:
# LightGBM parameters found by Bayesian optimization
clf = LGBMClassifier(
    nthread=4,
    n_estimators=10000,
    learning_rate=0.02,
    num_leaves=34,
    colsample_bytree=0.9497036,
    subsample=0.8715623,
    max_depth=8,
    reg_alpha=0.041545473,
    reg_lambda=0.0735294,
    min_split_gain=0.0222415,
    min_child_weight=39.3259775,
    silent=-1,
    verbose=-1, )

clf.fit(x_train, y_train, eval_set=[(x_train, y_train)],
        eval_metric='auc', verbose=200)


[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
[200]	training's auc: 0.797198	training's binary_logloss: 0.234742
[400]	training's auc: 0.819484	training's binary_logloss: 0.225443
[600]	training's auc: 0.83453	training's binary_logloss: 0.219295
[800]	training's auc: 0.846823	training's binary_logloss: 0.214246
[1000]	training's auc: 0.857045	training's binary_logloss: 0.2099
[1200]	training's auc: 0.866444	training's binary_logloss: 0.205814
[1400]	training's auc: 0.874943	training's binary_logloss: 0.201943
[1600]	training's auc: 0.8831	training's binary_logloss: 0.198133
[1800]	training's auc: 0.89018	training's binary_logloss: 0.194644
[2000]	training's auc: 0.897027	training's binary_logloss: 0.191207
[2200]	training's auc: 0.903096	training's binary_logloss: 0.188037
[2400]	training's auc: 0.909046	training's binary_logloss: 0.184842
[2600]	training's auc: 0.914644	training's binary_logloss: 0.181694
[2800]	

LGBMClassifier(colsample_bytree=0.9497036, learning_rate=0.02, max_depth=8,
               min_child_weight=39.3259775, min_split_gain=0.0222415,
               n_estimators=10000, nthread=4, num_leaves=34,
               reg_alpha=0.041545473, reg_lambda=0.0735294, silent=-1,
               subsample=0.8715623, verbose=-1)

# 5. Submission

In [ ]:
submission = pd.DataFrame({
    "SK_ID_CURR": test.SK_ID_CURR,
    "TARGET": clf.predict_proba(x_test)[:,1]
})
submission.to_csv("submission.csv", index = None)